In [ ]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]

# 文書番号をall_numに読み込み
all_num = np.array(all_data_pd["seq"], dtype= "int64")
# print(all_X)
print(all_num)

         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


all_X = all_X[::, :18]
#評価指標の指定
scoring = ["f1", "accuracy", "precision", "recall"]
for c in [0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0]:
    for g in [1.0,0.1,0.01,0.001,0.0001,0.00001]:
        #パイプラインの作成(標準化、SVM)
        pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
        
        #層化10分割交差検証のための分割の実施
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
        #層化10分割考査検証の実施
        scores = cross_validate(pipe, all_X, all_y, scoring=scoring, cv=kf)
        
        #評価値の出力
        print("C:%s      gamma:%s" %(c,g))
        for key,value in scores.items():
            print("{}:{:.4f}+/-{:.4f}".format(key, value.mean(), value.std()))
        print("\n")
        
        

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

for f in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14, 15, 16, 17, 18]:
    #使用する特徴量の指定
    all_X = all_data[:9813,3:33]
    all_X = all_X[::, :f]

    for c in [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]:
        for g in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001]:
            i = 1
            #十回分のそれぞれのスコアを格納する空のlistを作成
            accuracy_train = []
            f1_train = []
            precision_train = []
            recall_train = []

            accuracy_test = []
            f1_test = []
            precision_test = []
            recall_test = []
            
            #パイプラインの作成
            pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
        
            #層化10-fold交差検証の開始
            kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
            for train_index, test_index in kf.split(all_X, all_y):
                #繰り返し毎に訓練データとテストデータと文書番号の分割
                train_X, test_X = all_X[train_index], all_X[test_index]
                train_y, test_y = all_y[train_index], all_y[test_index]
                train_num, test_num = all_num[train_index], all_num[test_index]
                #訓練データのラベル1のsmote前のデータ数の出力
                #print("train_X(label=1): %s" %train_X[train_y == 1].shape[0])
                #print("train_X(label=0): %s" %train_X[train_y == 0].shape[0])
                
                #パイプを訓練データに適合
                pipe.fit(train_X, train_y)
                #訓練データを入力した時の予測ラベルをpre_label_trainに読み込む
                pre_label_train = pipe.predict(train_X)
                
                #訓練データにおける評価の実施
                accuracy_train.append(accuracy_score(train_y, pre_label_train))
                f1_train.append(f1_score(train_y, pre_label_train))
                precision_train.append(precision_score(train_y, pre_label_train))
                recall_train.append(recall_score(train_y, pre_label_train))

                #テストデータを入力した時の予測ラベルをpre_label_testに読み込む
                pre_label_test = pipe.predict(test_X)
                
                #テストデータにおける評価の実施
                accuracy_test.append(accuracy_score(test_y, pre_label_test))
                f1_test.append(f1_score(test_y, pre_label_test))
                precision_test.append(precision_score(test_y, pre_label_test))
                recall_test.append(recall_score(test_y, pre_label_test))
                
                
                #予測ラベルに対応する文書番号の結合
                V_test_num = test_num.reshape(len(test_num), 1)
                V_pre_label_test = pre_label_test.reshape(len(pre_label_test), 1)
                num_and_y = np.hstack((V_test_num, V_pre_label_test))
                
                #交差検証の何個目を行っているのかの出力
                #print(i)
                
                #csvファイルに出力
                np.savetxt("%d_direct_num_and_prelabel.csv" %i, num_and_y, fmt="%3.f", delimiter="\t", header="文書番号\t予測ラベル")
                i = i + 1

            #十回のスコアの平均と標準偏差の出力
            print("\n")
            #訓練データに関する結果
            print("変数の数:%s      C:%s      gamma:%s" %(f,c,g))
            print("accuracy_train: %.4f +- %.4f" %(np.mean(accuracy_train), np.std(accuracy_train)))
            print("f1_train: %.4f +- %.4f" %(np.mean(f1_train), np.std(f1_train)))
            print("precision_train: %.4f +- %.4f" %(np.mean(precision_train), np.std(precision_train)))
            print("recall_train: %.4f +- %.4f" %(np.mean(recall_train), np.std(recall_train)))
            
            #テストデータに関する結果
            print("accuracy_test: %.4f +- %.4f" %(np.mean(accuracy_test), np.std(accuracy_test)))
            print("f1_test: %.4f +- %.4f" %(np.mean(f1_test), np.std(f1_test)))
            print("precision_test: %.4f +- %.4f" %(np.mean(precision_test), np.std(precision_test)))
            print("recall_test: %.4f +- %.4f" %(np.mean(recall_test), np.std(recall_test)))
            print("list_shape: %s" %len(accuracy_train))
            print("\n")



変数の数:2      C:0.1      gamma:1.0
accuracy_train: 0.8818 +- 0.0009
f1_train: 0.4330 +- 0.0067
precision_train: 0.7035 +- 0.0067
recall_train: 0.3129 +- 0.0067
accuracy_test: 0.8813 +- 0.0058
f1_test: 0.4297 +- 0.0306
precision_test: 0.7043 +- 0.0567
recall_test: 0.3107 +- 0.0311
list_shape: 10




変数の数:2      C:0.1      gamma:0.1
accuracy_train: 0.8767 +- 0.0006
f1_train: 0.3314 +- 0.0047
precision_train: 0.7611 +- 0.0044
recall_train: 0.2118 +- 0.0036
accuracy_test: 0.8764 +- 0.0040
f1_test: 0.3290 +- 0.0374
precision_test: 0.7605 +- 0.0498
recall_test: 0.2111 +- 0.0318
list_shape: 10




変数の数:2      C:0.1      gamma:0.01
accuracy_train: 0.8696 +- 0.0002
f1_train: 0.2054 +- 0.0030
precision_train: 0.8528 +- 0.0068
recall_train: 0.1168 +- 0.0020
accuracy_test: 0.8698 +- 0.0021
f1_test: 0.2056 +- 0.0280
precision_test: 0.8591 +- 0.0388
recall_test: 0.1172 +- 0.0186
list_shape: 10




/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)




変数の数:2      C:0.1      gamma:0.001
accuracy_train: 0.8579 +- 0.0002
f1_train: 0.0313 +- 0.0026
precision_train: 0.9573 +- 0.0147
recall_train: 0.0159 +- 0.0013
accuracy_test: 0.8578 +- 0.0014
f1_test: 0.0305 +- 0.0161
precision_test: 0.8667 +- 0.3055
recall_test: 0.0155 +- 0.0083
list_shape: 10




/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

...

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio



変数の数:5      C:0.1      gamma:1e-05
accuracy_train: 0.8557 +- 0.0000
f1_train: 0.0000 +- 0.0000
precision_train: 0.0000 +- 0.0000
recall_train: 0.0000 +- 0.0000
accuracy_test: 0.8557 +- 0.0004
f1_test: 0.0000 +- 0.0000
precision_test: 0.0000 +- 0.0000
recall_test: 0.0000 +- 0.0000
list_shape: 10




変数の数:5      C:1.0      gamma:1.0
accuracy_train: 0.8839 +- 0.0004
f1_train: 0.4470 +- 0.0035
precision_train: 0.7144 +- 0.0041
recall_train: 0.3253 +- 0.0039
accuracy_test: 0.8806 +- 0.0043
f1_test: 0.4295 +- 0.0220
precision_test: 0.6950 +- 0.0462
recall_test: 0.3121 +- 0.0251
list_shape: 10




変数の数:5      C:1.0      gamma:0.1
accuracy_train: 0.8781 +- 0.0007
f1_train: 0.3692 +- 0.0064
precision_train: 0.7282 +- 0.0041
recall_train: 0.2473 +- 0.0054
accuracy_test: 0.8777 +- 0.0037
f1_test: 0.3652 +- 0.0320
precision_test: 0.7329 +- 0.0552
recall_test: 0.2450 +- 0.0319
list_shape: 10




変数の数:5      C:1.0      gamma:0.01
accuracy_train: 0.8752 +- 0.0005
f1_train: 0.2983 +- 0.0051
precisio

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio



変数の数:5      C:1.0      gamma:1e-05
accuracy_train: 0.8557 +- 0.0000
f1_train: 0.0000 +- 0.0000
precision_train: 0.0000 +- 0.0000
recall_train: 0.0000 +- 0.0000
accuracy_test: 0.8557 +- 0.0004
f1_test: 0.0000 +- 0.0000
precision_test: 0.0000 +- 0.0000
recall_test: 0.0000 +- 0.0000
list_shape: 10




変数の数:5      C:10.0      gamma:1.0
accuracy_train: 0.8882 +- 0.0006
f1_train: 0.4579 +- 0.0038
precision_train: 0.7620 +- 0.0085
recall_train: 0.3273 +- 0.0044
accuracy_test: 0.8782 +- 0.0048
f1_test: 0.4096 +- 0.0254
precision_test: 0.6832 +- 0.0419
recall_test: 0.2931 +- 0.0229
list_shape: 10




変数の数:5      C:10.0      gamma:0.1
accuracy_train: 0.8797 +- 0.0012
f1_train: 0.3912 +- 0.0102
precision_train: 0.7248 +- 0.0060
recall_train: 0.2680 +- 0.0089
accuracy_test: 0.8784 +- 0.0045
f1_test: 0.3835 +- 0.0351
precision_test: 0.7180 +- 0.0512
recall_test: 0.2634 +- 0.0339
list_shape: 10




変数の数:5      C:10.0      gamma:0.01
accuracy_train: 0.8760 +- 0.0005
f1_train: 0.3159 +- 0.0056
preci